In [1]:
import numpy as np
import pandas as pd

import math

### Load all nflscrapR rows

In [2]:
seasons = list(range(2009,2019)) # creates list of years through 2018

# merge seasons into one df
df = pd.DataFrame()
for season in seasons:
    path = './../data/nflscrapR-data/play_by_play_data/regular_season/reg_pbp_'+str(season)+'.csv'
    season_df = pd.read_csv(path)
    season_df['season'] = season
    df = pd.concat([df, season_df], axis=0)
    path = './../data/nflscrapR-data/play_by_play_data/post_season/post_pbp_'+str(season)+'.csv'
    post_season_df = pd.read_csv(path)
    post_season_df['season'] = season
    df = pd.concat([df, post_season_df], axis=0)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (42,166,167,174,175,178,179,182,183,188,189,190,191,203,204,205,231,232,233,238,240,241) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (37,38,39,42,46,47,166,167,168,169,172,173,174,175,176,177,178,179,182,183,184,185,188,189,190,191,192,193,194,195,198,199,203,204,205,224,225,226,227,231,232,233,238,240,241,244,245,246,250) have mixed types. Speci

In [3]:
# jacksonville has different abbreviations
df['posteam']=df['posteam'].replace('JAC','JAX')

conf_teams = ['AFC','NFC','IRV','APR','NPR','RIC','SANS']
df = df.loc[~df['posteam'].isin(conf_teams)]
print(len(df))
# print(list(df))

472337


In [4]:
attr = pd.read_csv('player_attr.csv')
print(attr.head())

   age  av                                       draft  exp  height  \
0   26   0  Tampa Bay Buccaneers / 7th / 231st pick /     4    74.0   
1   24   1                                         NaN    2    68.0   
2   24   0                                         NaN    0    77.0   
3   32   1          New York Jets / 1st / 27th pick /     9    78.0   
4   31   0    Kansas City Chiefs / 4th / 107th pick /     8    76.0   

                  id            name  pos salary  season  short_name team  \
0  2009ARIH.Abdullah  Hamza Abdullah  NaN    NaN    2009  H.Abdullah  ARI   
1     2009ARIM.Adams   Michael Adams   DB    NaN    2009     M.Adams  ARI   
2     2009ARIJ.Banks     Jason Banks  NaN    NaN    2009     J.Banks  ARI   
3     2009ARIA.Becht   Anthony Becht   TE    NaN    2009     A.Becht  ARI   
4    2009ARIM.Beisel    Monty Beisel  NaN    NaN    2009    M.Beisel  ARI   

   weight  
0     216  
1     178  
2     300  
3     270  
4     246  


### Merge in QBs that have 20 pass attempts in the nflscrapR dataset and are labeled QB in pro-football-reference

In [5]:
qbs = df.groupby(['passer_player_id'])['epa'].count().reset_index()
qb_ids = qbs.loc[qbs.epa>=20]
qb_ids = list(qb_ids.passer_player_id.values)
qb_col = df[['season','posteam','passer_player_id','passer_player_name']]
qb_col = qb_col.loc[qb_col['passer_player_id'].isin(qb_ids)]
qb_col['Prob Position'] = 'QB'
qb_col['season'] = qb_col.season.astype(str)
qb_col['id'] = qb_col['season'] + qb_col['posteam'] + qb_col['passer_player_name']

print(len(qb_col))
qb_col = qb_col.drop_duplicates(subset=['id'])
print(len(qb_col))
print(qb_col.head(10))

197611
717
    season posteam passer_player_id passer_player_name Prob Position  \
1     2009     PIT       00-0022924   B.Roethlisberger            QB   
6     2009     TEN       00-0003292          K.Collins            QB   
180   2009     CLE       00-0025409            B.Quinn            QB   
190   2009     MIN       00-0005106            B.Favre            QB   
332   2009     MIN       00-0024279          T.Jackson            QB   
351   2009      NO       00-0020531            D.Brees            QB   
359   2009     DET       00-0026498         M.Stafford            QB   
534   2009     DAL       00-0021678             T.Romo            QB   
546   2009      TB       00-0022177         B.Leftwich            QB   
705   2009     HOU       00-0022787           M.Schaub            QB   

                          id  
1    2009PITB.Roethlisberger  
6           2009TENK.Collins  
180           2009CLEB.Quinn  
190           2009MINB.Favre  
332         2009MINT.Jackson  
351       

In [6]:
# print(len(attr))
att_qb = attr.loc[attr['pos']=='QB']

new = pd.merge(how='left',left=qb_col,right=att_qb,on=['id','id'])
# print(new.tail(50))

print(len(new))
sure_qbs = new.loc[(new['Prob Position']=='QB') & (new['pos']=='QB')]
print(len(sure_qbs))

sure_qbs = sure_qbs.rename(columns={'season_x':'season'})
sure_qbs = sure_qbs.drop(columns=['season_y', 'Prob Position'])

print(sure_qbs.head(5))

717
524
  season posteam passer_player_id passer_player_name                       id  \
0   2009     PIT       00-0022924   B.Roethlisberger  2009PITB.Roethlisberger   
1   2009     TEN       00-0003292          K.Collins         2009TENK.Collins   
2   2009     CLE       00-0025409            B.Quinn           2009CLEB.Quinn   
3   2009     MIN       00-0005106            B.Favre           2009MINB.Favre   
5   2009      NO       00-0020531            D.Brees            2009NOD.Brees   

    age    av                                     draft   exp  height  \
0  27.0  14.0  Pittsburgh Steelers / 1st / 11th pick /    5.0    77.0   
1  37.0   2.0     Carolina Panthers / 1st / 5th pick /   14.0    77.0   
2  25.0   2.0     Cleveland Browns / 1st / 22nd pick /    2.0    76.0   
3  40.0  16.0      Atlanta Falcons / 2nd / 33rd pick /   18.0    74.0   
5  30.0  16.0   San Diego Chargers / 2nd / 32nd pick /    8.0    72.0   

                 name pos salary        short_name team  weight  


### Same for rbs with more than 50 rushes

In [7]:
rbs = df.groupby(['rusher_player_id'])['epa'].count().reset_index()
rb_ids = rbs.loc[rbs.epa>=50]
rb_ids = list(rb_ids.rusher_player_id.values)
# drop qbs
rb_ids = list(set(rb_ids)-set(qb_ids))
rb_col = df[['season','posteam','rusher_player_id','rusher_player_name']]
rb_col = rb_col.loc[rb_col['rusher_player_id'].isin(rb_ids)]
rb_col['Prob Position'] = 'RB'
rb_col['season'] = rb_col.season.astype(str)
rb_col['id'] = rb_col['season'] + rb_col['posteam'] + rb_col['rusher_player_name']

print(len(rb_col))
rb_col = rb_col.drop_duplicates(subset=['id'])
print(len(rb_col))
print(rb_col.head(50))

120881
1277
     season posteam rusher_player_id rusher_player_name Prob Position  \
2      2009     PIT       00-0022250           W.Parker            RB   
5      2009     TEN       00-0026164          C.Johnson            RB   
21     2009     PIT       00-0026163       R.Mendenhall            RB   
28     2009     TEN       00-0024260            L.White            RB   
139    2009     PIT       00-0022217            M.Moore            RB   
179    2009     CLE       00-0019693            J.Lewis            RB   
181    2009     CLE       00-0023176           J.Cribbs            RB   
188    2009     MIN       00-0025394         A.Peterson            RB   
204    2009     MIN       00-0026998           P.Harvin            RB   
234    2009     MIN       00-0021314           C.Taylor            RB   
353    2009      NO       00-0024011             M.Bell            RB   
354    2009      NO       00-0024217             R.Bush            RB   
358    2009     DET       00-0026204   

In [8]:
att_rb = attr.loc[attr['pos']=='RB']

new = pd.merge(how='left',left=rb_col,right=att_rb,on=['id','id'])

print(len(new))
sure_rbs = new.loc[(new['Prob Position']=='RB') & (new['pos']=='RB')]
print(len(sure_rbs))

sure_rbs = sure_rbs.rename(columns={'season_x':'season'})
sure_rbs = sure_rbs.drop(columns=['season_y', 'Prob Position'])
print(sure_rbs.tail(5))

1277
698
     season posteam rusher_player_id rusher_player_name                id  \
1262   2018     PIT       00-0034331          J.Samuels  2018PITJ.Samuels   
1264   2018     NYJ       00-0033295          E.McGuire  2018NYJE.McGuire   
1265   2018     DET       00-0031665           Z.Zenner   2018DETZ.Zenner   
1267   2018      SF       00-0034115           J.Wilson    2018SFJ.Wilson   
1272   2018      LA       00-0029854         C.Anderson  2018LAC.Anderson   

       age   av                                      draft  exp  height  \
1262  22.0  3.0  Pittsburgh Steelers / 5th / 165th pick /   0.0    72.0   
1264  24.0  3.0        New York Jets / 6th / 188th pick /   1.0    70.0   
1265  27.0  2.0                                        NaN  3.0    71.0   
1267  23.0  2.0                                        NaN  0.0    72.0   
1272  27.0  2.0                                        NaN  5.0    68.0   

                name pos    salary  short_name team  weight  
1262  Jaylen Sa

### Same for wide receivers with more than 50 receptions

In [9]:
wrs = df.groupby(['receiver_player_id'])['epa'].count().reset_index()
wr_ids = wrs.loc[wrs.epa>=50]
wr_ids = list(wr_ids.receiver_player_id.values)
# drop qbs & rbs
wr_ids = list(set(wr_ids)-set(qb_ids))
wr_ids = list(set(wr_ids)-set(rb_ids))
wr_col = df[['season','posteam','receiver_player_id','receiver_player_name']]
wr_col = wr_col.loc[wr_col['receiver_player_id'].isin(wr_ids)]
wr_col['Prob Position'] = 'WR/TE'
wr_col['season'] = wr_col.season.astype(str)
wr_col['id'] = wr_col['season'] + wr_col['posteam'] + wr_col['receiver_player_name']

print(len(wr_col))
wr_col = wr_col.drop_duplicates(subset=['id'])
print(len(wr_col))
# print(wr_col.head(50))

134176
2331


In [10]:
att_wr = attr.loc[attr['pos']=='WR']

new = pd.merge(how='left',left=wr_col,right=att_wr,on=['id','id'])

print(len(new))
sure_wrs = new.loc[(new['Prob Position']=='WR/TE') & (new['pos']=='WR')]
print(len(sure_wrs))

sure_wrs = sure_wrs.rename(columns={'season_x':'season'})
sure_wrs = sure_wrs.drop(columns=['season_y', 'Prob Position'])
print(sure_wrs.tail(5))

2333
1156
     season posteam receiver_player_id receiver_player_name  \
2323   2018     NYG         00-0032966            C.Coleman   
2324   2018     DET         00-0031326          B.Ellington   
2325   2018     BUF         00-0029388           D.Thompson   
2326   2018     MIA         00-0030516             B.Butler   
2329   2018     PIT         00-0031782             E.Rogers   

                      id   age   av  \
2323    2018NYGC.Coleman  24.0  1.0   
2324  2018DETB.Ellington  27.0  1.0   
2325   2018BUFD.Thompson  29.0  0.0   
2326     2018MIAB.Butler  28.0  0.0   
2329     2018PITE.Rogers  26.0  1.0   

                                          draft  exp  height             name  \
2323      Cleveland Browns / 1st / 15th pick /   2.0    71.0    Corey Coleman   
2324  San Francisco 49ers / 4th / 106th pick /   4.0    69.0  Bruce Ellington   
2325                                        NaN  6.0    72.0  Deonte Thompson   
2326      Oakland Raiders / 7th / 209th pick /   5.0

### And tight ends

In [11]:
att_te = attr.loc[attr['pos']=='TE']

sure_tes = pd.merge(how='left',left=wr_col,right=att_te,on=['id','id'])

print(len(new))
sure_tes = sure_tes.loc[(sure_tes['Prob Position']=='WR/TE') & (sure_tes['pos']=='TE')]
print(len(sure_tes))

sure_tes = sure_tes.rename(columns={'season_x':'season'})
sure_tes = sure_tes.drop(columns=['season_y', 'Prob Position'])
print(sure_tes.tail(5))

2333
656
     season posteam receiver_player_id receiver_player_name  \
2297   2018     OAK         00-0028097              L.Smith   
2306   2018     MIA         00-0032117            N.O'Leary   
2312   2018     JAX         00-0032062               B.Bell   
2313   2018     SEA         00-0027675            E.Dickson   
2329   2018     ARI         00-0027139           J.Phillips   

                     id   age   av  \
2297     2018OAKL.Smith  31.0  1.0   
2306   2018MIAN.O'Leary  26.0  1.0   
2312      2018JAXB.Bell  27.0  0.0   
2313   2018SEAE.Dickson  31.0  1.0   
2329  2018ARIJ.Phillips  31.0  0.0   

                                           draft  exp  height           name  \
2297  New England Patriots / 5th / 159th pick /   7.0    78.0      Lee Smith   
2306         Buffalo Bills / 6th / 194th pick /   3.0    75.0   Nick O'Leary   
2312   San Francisco 49ers / 4th / 117th pick /   3.0    78.0     Blake Bell   
2313       Baltimore Ravens / 3rd / 70th pick /   8.0    76.0  

### Put them all together

In [12]:
sure_players = None
sure_players = pd.concat([sure_qbs,sure_rbs],axis=0,sort=False)
sure_players = pd.concat([sure_players,sure_wrs],axis=0,sort=False)
sure_players = pd.concat([sure_players,sure_tes],axis=0,sort=False)

print(sure_players.tail(5))

     season posteam passer_player_id passer_player_name                 id  \
2297   2018     OAK              NaN                NaN     2018OAKL.Smith   
2306   2018     MIA              NaN                NaN   2018MIAN.O'Leary   
2312   2018     JAX              NaN                NaN      2018JAXB.Bell   
2313   2018     SEA              NaN                NaN   2018SEAE.Dickson   
2329   2018     ARI              NaN                NaN  2018ARIJ.Phillips   

       age   av                                       draft  exp  height  \
2297  31.0  1.0  New England Patriots / 5th / 159th pick /   7.0    78.0   
2306  26.0  1.0         Buffalo Bills / 6th / 194th pick /   3.0    75.0   
2312  27.0  0.0   San Francisco 49ers / 4th / 117th pick /   3.0    78.0   
2313  31.0  1.0       Baltimore Ravens / 3rd / 70th pick /   8.0    76.0   
2329  31.0  0.0        Dallas Cowboys / 6th / 208th pick /   9.0    77.0   

               name pos      salary  short_name team  weight rusher_player

In [13]:
qb_ids = list(sure_players.passer_player_id.unique())
rb_ids = list(sure_players.rusher_player_id.unique())
wr_ids = list(sure_players.receiver_player_id.unique())

sure_ids = list(set(qb_ids+rb_ids+wr_ids))
print(len(sure_ids))

852


In [14]:
# runs = df.loc[df.play_type=='run']
# print(len(runs))

# found = runs.loc[runs.rusher_player_id.isin(sure_ids)]
# print(len(found))

# 129201 out of 139328 -- 93% rush plays

In [15]:
# _qbs = df.loc[df.play_type=='pass']
# print(len(_qbs))

# found = _qbs.loc[_qbs.passer_player_id.isin(sure_ids)]
# print(len(found))

#175,000 receivers out of 196,484 plays -- 89.3%
#195,247 out of 196,484 passers -- 99.3%

### Merge in players that don't have a position listed on pro-football-reference

In [16]:
pos_list = ['DB', 'TE', 'ROLB', 'WR', 'LT', 'RT', 'DE', 'LB', 'DT', 'P', 'LOLB', 'RB', 'FS', 'FB', 'K', 'QB', 'RG', 'RCB', 'NT', 'LCB', 'C', 'LG', 'MLB', 'SS', 'S', 'CB', 'LDT', 'T', 'RILB', 'RDT', 'LILB', 'G', 'DL']
nconf = attr.loc[~attr['pos'].isin(pos_list)]

# merge in players without listed position
print(len(qb_col))
nconf_qb = qb_col.loc[~qb_col.passer_player_id.isin(sure_ids)]
print(len(nconf_qb))
maybe_qbs = pd.merge(how='left',left=nconf_qb,right=nconf,on=['id','id'])

maybe_qbs.pos = 'QB'
print(len(new))

maybe_qbs.dropna(subset=['name'],inplace=True)

maybe_ids = list(maybe_qbs.passer_player_id.unique())

maybe_qbs = maybe_qbs.rename(columns={'season_x':'season'})
maybe_qbs = maybe_qbs.drop(columns=['season_y', 'Prob Position'])
print(maybe_qbs.tail(5))

717
27
2333
   season posteam passer_player_id passer_player_name                id   age  \
20   2016     CHI       00-0031076            D.Fales    2016CHID.Fales  26.0   
21   2016     OAK       00-0032893             C.Cook     2016OAKC.Cook  23.0   
23   2017     MIA       00-0031076            D.Fales    2017MIAD.Fales  27.0   
24   2017     PHI       00-0032792          N.Sudfeld  2017PHIN.Sudfeld  24.0   
26   2018     PHI       00-0032792          N.Sudfeld  2018PHIN.Sudfeld  25.0   

     av                                      draft  exp  height          name  \
20  0.0        Chicago Bears / 6th / 183rd pick /   2.0    74.0   David Fales   
21  0.0      Oakland Raiders / 4th / 100th pick /   0.0    76.0   Connor Cook   
23  1.0        Chicago Bears / 6th / 183rd pick /   3.0    74.0   David Fales   
24  1.0  Washington Redskins / 6th / 187th pick /   0.0    78.0  Nate Sudfeld   
26  0.0  Washington Redskins / 6th / 187th pick /   1.0    78.0  Nate Sudfeld   

   pos    sala

In [17]:
print(len(rb_col))
nconf_rb = rb_col.loc[~rb_col.rusher_player_id.isin(sure_ids)]
print(len(nconf_rb))
nconf_rb = nconf_rb.loc[~nconf_rb.rusher_player_id.isin(maybe_ids)]
print(len(nconf_rb))
maybe_rbs = pd.merge(how='left',left=nconf_rb,right=nconf,on=['id','id'])

maybe_rbs.pos = 'RB'

maybe_rbs.dropna(subset=['name'],inplace=True)

print(len(maybe_ids))
maybe_ids = list(set(maybe_ids + list(maybe_rbs.rusher_player_id)))
print(len(maybe_ids))

maybe_rbs = maybe_rbs.rename(columns={'season_x':'season'})
maybe_rbs = maybe_rbs.drop(columns=['season_y', 'Prob Position'])
print(maybe_rbs.tail(5))

1277
192
192
6
43
    season posteam rusher_player_id rusher_player_name                   id  \
185   2018     ARI       00-0034681          C.Edmonds     2018ARIC.Edmonds   
186   2018     ATL       00-0034378            I.Smith       2018ATLI.Smith   
187   2018     WAS       00-0031102            K.Bibbs       2018WASK.Bibbs   
188   2018     CHI       00-0029795       B.Cunningham  2018CHIB.Cunningham   
191   2018      GB       00-0031102            K.Bibbs        2018GBK.Bibbs   

      age   av                                    draft  exp  height  \
185  22.0  2.0  Arizona Cardinals / 4th / 134th pick /   0.0    69.0   
186  23.0  3.0    Atlanta Falcons / 4th / 126th pick /   0.0    69.0   
187  25.0  1.0                                      NaN  4.0    70.0   
188  28.0  0.0                                      NaN  5.0    70.0   
191  25.0  0.0                                      NaN  4.0    70.0   

                 name pos    salary    short_name team  weight  
185     C

In [18]:
print(len(wr_col))
nconf_wr = wr_col.loc[~wr_col.receiver_player_id.isin(sure_ids)]
print(len(nconf_wr))
nconf_wr = nconf_wr.loc[~nconf_wr.receiver_player_id.isin(maybe_ids)]
print(len(nconf_wr))
maybe_wrs = pd.merge(how='left',left=nconf_wr,right=nconf,on=['id','id'])
# print(new.tail(10))

maybe_wrs.pos = 'WR/TE'
print('maybe wrs len', len(maybe_wrs))
maybe_wrs.dropna(subset=['name'],inplace=True)
print(len(maybe_wrs))

print(len(maybe_ids))
maybe_ids = list(set(maybe_ids + list(maybe_wrs.receiver_player_id)))
print(len(maybe_ids))

maybe_wrs = maybe_wrs.rename(columns={'season_x':'season'})
maybe_wrs = maybe_wrs.drop(columns=['season_y', 'Prob Position'])
print(maybe_wrs.tail(5))

2331
132
132
maybe wrs len 132
21
43
54
    season posteam receiver_player_id receiver_player_name                 id  \
104   2015     ATL         00-0029848           N.Williams  2015ATLN.Williams   
116   2016     ATL         00-0029848           N.Williams  2016ATLN.Williams   
123   2017     ATL         00-0029848           N.Williams  2017ATLN.Williams   
129   2018     TEN         00-0029848           N.Williams  2018TENN.Williams   
131   2018      LA         00-0029848           N.Williams   2018LAN.Williams   

      age   av draft  exp  height           name    pos    salary  short_name  \
104  25.0  1.0   NaN  2.0    70.0  Nick Williams  WR/TE  $510,000  N.Williams   
116  26.0  1.0   NaN  3.0    70.0  Nick Williams  WR/TE  $141,176  N.Williams   
123  27.0  0.0   NaN  4.0    70.0  Nick Williams  WR/TE  $690,000  N.Williams   
129  28.0  0.0   NaN  5.0    70.0  Nick Williams  WR/TE  $697,065  N.Williams   
131  28.0  0.0   NaN  5.0    70.0  Nick Williams  WR/TE  $511,176  N

### Put all that together

In [19]:
maybe_players = None
maybe_players = pd.concat([maybe_qbs,maybe_rbs],axis=0,sort=False)
maybe_players = pd.concat([maybe_players,maybe_wrs],axis=0,sort=False)

sm_players = pd.concat([sure_players,maybe_players],axis=0,sort=False)
all_ids = sure_ids + maybe_ids
print(len(all_ids))

906


### Merge in players no matter the position listed

In [20]:
runs = df.loc[df.play_type=='run']

not_found = runs.loc[~runs.rusher_player_id.isin(all_ids)]

not_found.season = not_found.season.astype(str)
nf_col = not_found[['season','posteam','rusher_player_id','rusher_player_name']]
nf_col['id'] = nf_col.season + nf_col.posteam + nf_col.rusher_player_name

last_rbs = pd.merge(how='left',left=nf_col,right=attr,on=['id','id'])

last_rbs = last_rbs.rename(columns={'season_x':'season'})
last_rbs = last_rbs.drop(columns=['season_y'])
print(len(last_rbs))
last_rbs.dropna(subset=['name'],inplace=True)
print(len(last_rbs))

all_ids = list(set(all_ids + list(last_rbs.rusher_player_id.unique())))
print(len(all_ids))


5192
3848
1328


/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [21]:
passes = df.loc[df.play_type=='pass']

not_found = passes.loc[~passes.receiver_player_id.isin(all_ids)]

not_found.season = not_found.season.astype(str)
nf_col = not_found[['season','posteam','receiver_player_id','receiver_player_name']]
nf_col['id'] = nf_col.season + nf_col.posteam + nf_col.receiver_player_name

last_wrs = pd.merge(how='left',left=nf_col,right=attr,on=['id','id'])

last_wrs = last_wrs.rename(columns={'season_x':'season'})
last_wrs = last_wrs.drop(columns=['season_y'])

last_wrs.dropna(subset=['name'],inplace=True)

print(len(all_ids))
all_ids = list(set(all_ids + list(last_wrs.receiver_player_id.unique())))
print(len(all_ids))

1328
1834


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [22]:
not_found = passes.loc[~passes.passer_player_id.isin(all_ids)]

not_found.season = not_found.season.astype(str)
nf_col = not_found[['season','posteam','passer_player_id','passer_player_name']]
nf_col['id'] = nf_col.season + nf_col.posteam + nf_col.passer_player_name

last_qbs = pd.merge(how='left',left=nf_col,right=attr,on=['id','id'])

last_qbs = last_qbs.rename(columns={'season_x':'season'})
last_qbs = last_qbs.drop(columns=['season_y'])

last_qbs.dropna(subset=['name'],inplace=True)

print(len(all_ids))
all_ids = list(set(all_ids + list(last_qbs.passer_player_id.unique())))
print(len(all_ids))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


1834
1863


In [23]:
last_players = None
last_players = pd.concat([last_qbs,last_rbs],axis=0,sort=False)
last_players = pd.concat([last_players,last_wrs],axis=0,sort=False)

all_players = pd.concat([sm_players,last_players],axis=0,sort=False)

print(len(all_players))
print(len(all_ids))

13125
1863


### Cleanup and output file

In [24]:
output = all_players.drop(columns=['passer_player_name','rusher_player_name','receiver_player_name'])

print(output.head())

  season posteam passer_player_id                       id   age    av  \
0   2009     PIT       00-0022924  2009PITB.Roethlisberger  27.0  14.0   
1   2009     TEN       00-0003292         2009TENK.Collins  37.0   2.0   
2   2009     CLE       00-0025409           2009CLEB.Quinn  25.0   2.0   
3   2009     MIN       00-0005106           2009MINB.Favre  40.0  16.0   
5   2009      NO       00-0020531            2009NOD.Brees  30.0  16.0   

                                      draft   exp  height                name  \
0  Pittsburgh Steelers / 1st / 11th pick /    5.0    77.0  Ben Roethlisberger   
1     Carolina Panthers / 1st / 5th pick /   14.0    77.0       Kerry Collins   
2     Cleveland Browns / 1st / 22nd pick /    2.0    76.0         Brady Quinn   
3      Atlanta Falcons / 2nd / 33rd pick /   18.0    74.0         Brett Favre   
5   San Diego Chargers / 2nd / 32nd pick /    8.0    72.0          Drew Brees   

  pos salary        short_name team  weight rusher_player_id  \
0  Q

In [29]:
id_output = output.copy()

def get_id(passer, rusher, receiver):
    if isinstance(passer,str):
        return passer
    if isinstance(rusher,str):
        return rusher
    if isinstance(receiver,str):
        return receiver
    
id_output['player_id'] = id_output.apply(lambda row: get_id(row['passer_player_id'],row['rusher_player_id'],row['receiver_player_id']),axis=1)

def hand_fix(name, pos):
    print(name)
    print(name)
    print(name)
    print(name)
    print(name)
    problematic = {
        'Julian Edelman':'WR',
        'Randall Cobb':'WR',
        'Percy Harvin':'WR',
        'Ted Ginn':'WR',
        'Kyle Juszczyk':'FB',
        
    }
    if isinstance(name,str):
        if name in problematic.keys():
            return problematic[name]
    return pos
id_output = id_output.reset_index()
print(id_output.name)
id_output['pos'] = id_output.apply(lambda row: hand_fix(row['name'],row['pos']))

roster_output = id_output.copy()
roster_output.drop_duplicates(subset=['id'],keep='last',inplace=True)
roster_output = roster_output.drop(columns=['passer_player_id','rusher_player_id','receiver_player_id'])

roster_output.to_csv('./output/roster_info.csv',index=False)
print(roster_output.head())

id_output.drop_duplicates(subset=['player_id'],keep='last',inplace=True)

id_output = id_output.drop(columns=['passer_player_id','rusher_player_id','receiver_player_id'])

id_output = id_output[['player_id','pos','name','short_name']]

id_output.to_csv('./output/player_ids.csv',index=False)

0        Ben Roethlisberger
1             Kerry Collins
2               Brady Quinn
3               Brett Favre
4                Drew Brees
5          Matthew Stafford
6                 Tony Romo
7            Byron Leftwich
8               Matt Schaub
9              Mark Sanchez
10           Peyton Manning
11            David Garrard
12               Kyle Orton
13            Carson Palmer
14            Jake Delhomme
15           Donovan McNabb
16               Kevin Kolb
17               Matt Moore
18            Brodie Croyle
19               Joe Flacco
20                Matt Ryan
21          Chad Pennington
22              Kurt Warner
23               Shaun Hill
24          Matt Hasselbeck
25              Eli Manning
26           Jason Campbell
27            Aaron Rodgers
28               Jay Cutler
29                Tom Brady
                ...        
13095            Noah Brown
13096          Blake Jarwin
13097          Blake Jarwin
13098          Blake Jarwin
13099            Noa

KeyError: ('name', 'occurred at index index')